# Week 4 - Preprocessing.

We'll be getting the data ready to be fed into a model, taking steps such as transforming the data (aggregation), removing columns (Month Grouping and Month abbreviation, etc), and one hot encoding for variables like Land Border Region or Title of Authority that won't give us 30+ more variables. 

In [1]:
from _Setup import *

Installing dependencies from requirements.txt...
All dependencies installed successfully.


In [2]:
sector_train = pd.read_csv(sector_data_csv_path_train)
sector_train.head(5)


,Fiscal Year,Month Grouping,Month (abbv),Component,Land Border Region,Area of Responsibility,AOR (Abbv),Demographic,Citizenship,Title of Authority,Encounter Type,Encounter Count,Year-Date
0,2020,FYTD,OCT,Office of Field Operations,Northern Land Border,Boston Field Office,Boston,FMUA,BRAZIL,Title 8,Inadmissibles,2,2019-10-01
1,2020,FYTD,OCT,Office of Field Operations,Northern Land Border,Boston Field Office,Boston,FMUA,OTHER,Title 8,Inadmissibles,29,2019-10-01
2,2020,FYTD,OCT,Office of Field Operations,Northern Land Border,Boston Field Office,Boston,Single Adults,BRAZIL,Title 8,Inadmissibles,1,2019-10-01
3,2020,FYTD,OCT,Office of Field Operations,Northern Land Border,Boston Field Office,Boston,Single Adults,CANADA,Title 8,Inadmissibles,1031,2019-10-01
4,2020,FYTD,OCT,Office of Field Operations,Northern Land Border,Boston Field Office,Boston,Single Adults,"CHINA, PEOPLES REPUBLIC OF",Title 8,Inadmissibles,9,2019-10-01


In [3]:
# These columns should be captured in Year-Date and we remove abbreviated columns that are unnecessary.
columns_to_remove = ["Fiscal Year", "Month Grouping", "Month (abbv)", "AOR (Abbv)"]
sector_train_dropped = sector_train.drop(columns = columns_to_remove)

In [4]:
sector_train_dropped.head(5)

,Component,Land Border Region,Area of Responsibility,Demographic,Citizenship,Title of Authority,Encounter Type,Encounter Count,Year-Date
0,Office of Field Operations,Northern Land Border,Boston Field Office,FMUA,BRAZIL,Title 8,Inadmissibles,2,2019-10-01
1,Office of Field Operations,Northern Land Border,Boston Field Office,FMUA,OTHER,Title 8,Inadmissibles,29,2019-10-01
2,Office of Field Operations,Northern Land Border,Boston Field Office,Single Adults,BRAZIL,Title 8,Inadmissibles,1,2019-10-01
3,Office of Field Operations,Northern Land Border,Boston Field Office,Single Adults,CANADA,Title 8,Inadmissibles,1031,2019-10-01
4,Office of Field Operations,Northern Land Border,Boston Field Office,Single Adults,"CHINA, PEOPLES REPUBLIC OF",Title 8,Inadmissibles,9,2019-10-01


These dataframes are prepped for simple time series models that follow a continuous variable throughout time for a given variable. In this case, because we're working with Sector-level data, we aggregate by Area of Responsibility to see the total encounter count for each sector/field office each month. 

In [5]:
# Aggregating by 'Area of Responsibility' and 'Year-Date'
sector_train_aggregated = sector_train_dropped.groupby(["Area of Responsibility", "Year-Date"], as_index=False).agg({"Encounter Count": "sum"})
sector_train_aggregated.head()

,Area of Responsibility,Year-Date,Encounter Count
0,Atlanta Field Office,2019-10-01,1022
1,Atlanta Field Office,2019-11-01,762
2,Atlanta Field Office,2019-12-01,564
3,Atlanta Field Office,2020-01-01,527
4,Atlanta Field Office,2020-02-01,521


To add value here, we can also create parallel time series to incorporate other factors such as the percentage of a country included in the encounters, percentage of unaccompanied minors, or other factors that will be useful in USBCP's planning process.

In [6]:
# Aggregating total encounters from Mexico per group
mexico_encounters = sector_train_dropped[sector_train_dropped["Citizenship"] == "MEXICO"].groupby(["Area of Responsibility", "Year-Date"], as_index=False)["Encounter Count"].sum()
mexico_encounters.head()

,Area of Responsibility,Year-Date,Encounter Count
0,Atlanta Field Office,2019-10-01,25
1,Atlanta Field Office,2019-11-01,24
2,Atlanta Field Office,2019-12-01,22
3,Atlanta Field Office,2020-01-01,19
4,Atlanta Field Office,2020-02-01,43


In [7]:
sector_train_aggregated = sector_train_aggregated.merge(mexico_encounters, on=["Area of Responsibility", "Year-Date"], how="left", suffixes=("", " Mexico"))
sector_train_aggregated['Encounter Count Mexico'] = sector_train_aggregated['Encounter Count Mexico'].fillna(0)
sector_train_aggregated.head()

,Area of Responsibility,Year-Date,Encounter Count,Encounter Count Mexico
0,Atlanta Field Office,2019-10-01,1022,25.0
1,Atlanta Field Office,2019-11-01,762,24.0
2,Atlanta Field Office,2019-12-01,564,22.0
3,Atlanta Field Office,2020-01-01,527,19.0
4,Atlanta Field Office,2020-02-01,521,43.0


We can perform a similar operation to see the aggregated number of unaccompanied minors as well

In [8]:
# Aggregating total encounters from Mexico per group
unaccompanied_encounters = sector_train_dropped[sector_train_dropped["Demographic"] == "UC / Single Minors"].groupby(["Area of Responsibility", "Year-Date"], as_index=False)["Encounter Count"].sum()
unaccompanied_encounters.head()

,Area of Responsibility,Year-Date,Encounter Count
0,Atlanta Field Office,2020-01-01,3
1,Atlanta Field Office,2020-07-01,3
2,Atlanta Field Office,2020-09-01,3
3,Atlanta Field Office,2020-10-01,2
4,Atlanta Field Office,2020-11-01,2


Concatenate the number of encounter column to the aggregated dataframe

In [9]:
sector_train_aggregated = sector_train_aggregated.merge(unaccompanied_encounters, on=["Area of Responsibility", "Year-Date"], how="left", suffixes=("", " UA"))
sector_train_aggregated['Encounter Count UA'] = sector_train_aggregated['Encounter Count UA'].fillna(0)
sector_train_aggregated.head()

,Area of Responsibility,Year-Date,Encounter Count,Encounter Count Mexico,Encounter Count UA
0,Atlanta Field Office,2019-10-01,1022,25.0,0.0
1,Atlanta Field Office,2019-11-01,762,24.0,0.0
2,Atlanta Field Office,2019-12-01,564,22.0,0.0
3,Atlanta Field Office,2020-01-01,527,19.0,3.0
4,Atlanta Field Office,2020-02-01,521,43.0,0.0


These other columns can be used to add value in a parallel series LSTM or other more advanced machine learning method. 

# This section is a bit experimental. 

We'd also like to aggregate Encounter Count by date and Area of Responsiblity, making sure we don't lose information such as Demographic, Citizenship, Title of Authority, and Encounter Type, etc.

Making sure we know the starting df shape

In [10]:
sector_train_dropped.shape

(57644, 9)

Creating a copy so I don't mess with the original dataframe

In [11]:
sector_train_dropped_copy = sector_train_dropped
sector_train_dropped_copy.shape

(57644, 9)

If I can duplicate a row, then try to aggregate Encounter Count with respect to all the other columns to retain all relevant information, we can see if this aggregation code is working as intended. Then we can check the original dataframe to see if aggregation is even possible or if all rows are unique!

In [12]:
row_to_dup = sector_train_dropped_copy.loc[1]

df_dup = pd.concat([sector_train_dropped_copy] + [row_to_dup.to_frame().T], ignore_index= True)
df_dup

,Component,Land Border Region,Area of Responsibility,Demographic,Citizenship,Title of Authority,Encounter Type,Encounter Count,Year-Date
0,Office of Field Operations,Northern Land Border,Boston Field Office,FMUA,BRAZIL,Title 8,Inadmissibles,2,2019-10-01
1,Office of Field Operations,Northern Land Border,Boston Field Office,FMUA,OTHER,Title 8,Inadmissibles,29,2019-10-01
2,Office of Field Operations,Northern Land Border,Boston Field Office,Single Adults,BRAZIL,Title 8,Inadmissibles,1,2019-10-01
3,Office of Field Operations,Northern Land Border,Boston Field Office,Single Adults,CANADA,Title 8,Inadmissibles,1031,2019-10-01
4,Office of Field Operations,Northern Land Border,Boston Field Office,Single Adults,"CHINA, PEOPLES REPUBLIC OF",Title 8,Inadmissibles,9,2019-10-01
...,...,...,...,...,...,...,...,...,...
57640,U.S. Border Patrol,Southwest Land Border,Yuma Sector,UC / Single Minors,NICARAGUA,Title 8,Apprehensions,3,2023-12-01
57641,U.S. Border Patrol,Southwest Land Border,Yuma Sector,UC / Single Minors,OTHER,Title 8,Apprehensions,13,2023-12-01
57642,U.S. Border Patrol,Southwest Land Border,Yuma Sector,UC / Single Minors,PERU,Title 8,Apprehensions,32,2023-12-01
57643,U.S. Border Patrol,Southwest Land Border,Yuma Sector,UC / Single Minors,RUSSIA,Title 8,Apprehensions,1,2023-12-01


In [13]:
# Using this to ensure all of the correct columns to use in group_by function
sector_train_dropped_copy.columns

Index(['Component', 'Land Border Region', 'Area of Responsibility',
       'Demographic', 'Citizenship', 'Title of Authority', 'Encounter Type',
       'Encounter Count', 'Year-Date'],
      dtype='object')

In [14]:

group_columns = [
    'Component', 'Land Border Region', 'Area of Responsibility',
       'Demographic', 'Citizenship', 'Title of Authority', 'Encounter Type',
        'Year-Date'
]

# Aggregate Encounter Count by summing it where all other columns are the same
aggregated_df = sector_train_dropped_copy.groupby(group_columns, as_index=False).agg({
    "Encounter Count": "sum"  # Sum the encounter counts for duplicate groups
})

# Display result
print(aggregated_df)

                        Component     Land Border Region  \
0      Office of Field Operations   Northern Land Border   
1      Office of Field Operations   Northern Land Border   
2      Office of Field Operations   Northern Land Border   
3      Office of Field Operations   Northern Land Border   
4      Office of Field Operations   Northern Land Border   
...                           ...                    ...   
57639          U.S. Border Patrol  Southwest Land Border   
57640          U.S. Border Patrol  Southwest Land Border   
57641          U.S. Border Patrol  Southwest Land Border   
57642          U.S. Border Patrol  Southwest Land Border   
57643          U.S. Border Patrol  Southwest Land Border   

      Area of Responsibility         Demographic Citizenship  \
0        Boston Field Office  Accompanied Minors      BRAZIL   
1        Boston Field Office  Accompanied Minors      BRAZIL   
2        Boston Field Office  Accompanied Minors      BRAZIL   
3        Boston Field O

The aggregated dataframe looks to have successfully aggregated the duplicated row with the original row, as the row count has decreased by 1, back to 57644. Let's try this aggregation code on the original dataframe now!

In [15]:

# Aggregate Encounter Count by summing it where all other columns are the same
aggregated_df = sector_train_dropped.groupby(group_columns, as_index=False).agg({
    "Encounter Count": "sum"  # Sum the encounter counts for duplicate groups
})

# Display result
print(aggregated_df)

                        Component     Land Border Region  \
0      Office of Field Operations   Northern Land Border   
1      Office of Field Operations   Northern Land Border   
2      Office of Field Operations   Northern Land Border   
3      Office of Field Operations   Northern Land Border   
4      Office of Field Operations   Northern Land Border   
...                           ...                    ...   
57639          U.S. Border Patrol  Southwest Land Border   
57640          U.S. Border Patrol  Southwest Land Border   
57641          U.S. Border Patrol  Southwest Land Border   
57642          U.S. Border Patrol  Southwest Land Border   
57643          U.S. Border Patrol  Southwest Land Border   

      Area of Responsibility         Demographic Citizenship  \
0        Boston Field Office  Accompanied Minors      BRAZIL   
1        Boston Field Office  Accompanied Minors      BRAZIL   
2        Boston Field Office  Accompanied Minors      BRAZIL   
3        Boston Field O

The original dataframe started out with 57644 rows and after "aggregation", it remains the same, which indicates that aggregation of the data is not possible as each row is unique! 

In [ ]:
# We can save this dataframe with dropped variables. Commented out to prevent unnecessary overwrites.
# sector_train_dropped.to_csv(sector_train_dropped_csv_path, index = False)

# This section will be for prepping the data for more complex models like neural networks (LSTMs) that can take in much more columns/info compared to simple time series.